In [ ]:
#Functions for PACT data processing in transfer to Box
#import datatools

import os
# "dev" matches the name in ~/.aws/config
os.environ['AWS_PROFILE'] = 'dev'
os.environ['AWS_DEFAULT_PROFILE'] = 'dev'
os.environ['HTTP_PROXY'] = 'http://proxy.sandia.gov:80'
os.environ['HTTPS_PROXY'] = 'http://proxy.sandia.gov:80'
os.environ['REQUESTS_CA_BUNDLE'] = '/Users/jsstein/Documents/SNL_Root_CA.crt'
os.environ['AWS_CA_BUNDLE'] = '/Users/jsstein/Documents/SNL_Root_CA.crt'
from tqdm import tqdm

import boto3

s3 = boto3.resource('s3')
bucket = s3.Bucket('pvivdb-transfer')

In [ ]:
#Reads in module setup file and generates dictionaries and variables
def read_pact_modules():
    import pandas as pd
    modules = pd.read_csv('PACT_SNL_Outdoor_Modules_SETUP.csv')
    active_modules = modules.loc[modules['Active']=='Y',:]
    # Initialize dictionaries
    allsamples = {}
    allareas = {}
    module_types = {}

    # Loop through the DataFrame rows
    for index, row in active_modules.iterrows():
        pact_id = row['PACT_id']
        psel_id = row['PSEL_id']
        area = row['Area']
        module_type = row['Type']
        start_date = row['Start_date']
        end_date = row['End_date']
        notes = row['Notes']
        
        # Assign values to dictionaries
        allsamples[pact_id] = psel_id
        allareas[pact_id] = area
        module_types[pact_id] = module_type

    # Inverse the allsamples dictionary
    pvid_to_pact = {v: k for k, v in allsamples.items()}
    
    # Generate the batches dictionary
    # Initialize the batches dictionary
    batches = {}

    # Loop through the DataFrame rows
    for index, row in active_modules.iterrows():
        pact_id = row['PACT_id']
        
        # Check if the PACT_ID starts with 'P-' and extract the prefix
        if pact_id.startswith('P-'):
            prefix = pact_id[:6]  # Get the first 6 characters (e.g., 'P-0003')
            
            # Add the PACT_ID to the corresponding prefix list in the batches dictionary
            if prefix not in batches:
                batches[prefix] = []
            batches[prefix].append(pact_id)
    
    # Initialize the current_modules dictionary
    current_modules = {}
    
    # Loop through the active_modules DataFrame rows to populate current_modules
    for index, row in active_modules.iterrows():
        start_date = row['Start_date']
        #end_date = row['End_date']
        pact_id = row['PACT_id']
        #area = row['Area']
        #active = row['Active']
        
        current_modules[pact_id] = {'start': start_date, 'end': 'NAN'}
    
    # Initialize the batches dictionary
    batches2 = {}
    
    # Loop through the active_modules DataFrame rows to populate batches
    for index, row in active_modules.iterrows():
        pact_id = row['PACT_id']
        
        # Extract the prefix (first 6 characters)
        prefix = pact_id[:6]
        
        # Initialize the prefix in batches if not already present
        if prefix not in batches2:
            batches2[prefix] = {'modules': []}
        
        # Append the current PACT_ID to the corresponding prefix's modules list
        batches2[prefix]['modules'].append(pact_id)
    return active_modules, allsamples, allareas, module_types, batches, batches2

In [ ]:
#Generate Module Metadata json files
def make_module_metadata_json(path, batch, batches, allareas, module_types):
    mod = []
    for module in batches[batch]:
        mod_dict = {'module_id': module,
                    'module_area': allareas[module],
                    'module_type': module_types[module],
                    'days_indoors': [],
                    'days_censored': []}
        mod.append(mod_dict)
    with open(path, 'w') as outfile:
        json.dump(mod, outfile) 
    print(batch + ': Generated module-metadata.json')

# module_id = 'P-0888-01'
# path = '/Users/jsstein/bin/Box Sync copy/PACT - Data/'
# metadata_path = path + module_id[:6] + '-XX/Outdoor_SNL/data/metadata/module-metadata.json'

In [ ]:
# Generate site-metadata.json
def make_site_metadata_json(site_path, label, latitude, longitude, elevation, surface_tilt, surface_azimuth):
    site_dict = {"location":{"label":label, 
                             "latitude":latitude, 
                             "longitude":longitude, 
                             "elevation":elevation,
                             "surface_tilt":surface_tilt, 
                             "surface_azimuth":surface_azimuth}, 
                             "snow_days":[]}
    with open(site_path, 'w') as outfile:
        json.dump(site_dict, outfile) 
    print('Generated site-metadata.json')

In [ ]:
#Function to list modules in module_metadata.json

def modules_from_metadata(path, batch):
    import json
    modules = []
    with open(path, 'r') as file:
        data = json.load(file)
        for module_data in data:
            modules.append(module_data['module_id'])
    return modules

#Testing the function
# batch = 'P-0042'
# path = '/Users/jsstein/bin/Box Sync/PACT - Data/'
# module_list = modules_from_metadata(path, batch)
# module_list

In [ ]:
# Check if module is in metadata
def check_for_new_module(pact_id, path):
    #metadata_path = path + pact_id[:6] + '-XX/Outdoor_SNL/data/metadata/module-metadata.json'
    module_list = modules_from_metadata(path, pact_id[:6])
    if pact_id in module_list:
        return True
    else:
        return False

#Test function
# path = '/Users/jsstein/bin/Box Sync/PACT - Data/'
# pact_id = 'P-0042-065'
# check_for_new_module(pact_id, path)

In [ ]:
# Function to add a module to metadata.json
# Assumes that pact_id is in the main module list with 
def add_module_to_metadata(path, pact_id):
    with open(path, 'r') as file:
        data = json.load(file)
    if check_for_new_module(pact_id, path):
        print(pact_id + ' is in module-metadata.json')
    else:
        pact_dict = {'module_id':pact_id, 
                     'module_area':allareas[pact_id], 
                     'module_type': module_types[pact_id],
                     'days_indoors': [],
                     'days_censored': []}
        data.append(pact_dict)
        with open(path, 'w') as outfile:
            json.dump(data, outfile) 
        print(pact_id + ' added to module-metadata.json')
    return data

# path = '/Users/jsstein/bin/Box Sync/PACT - Data/'
# batch = 'P-0138'
# add_module_to_metadata(path, batch, 'P-0138-01')

In [ ]:
import json

def check_and_add_censor_condition(file_path, pact_id, start, end, comment):
    # Load the existing JSON data from the file
    with open(file_path, 'r') as file:
        data = json.load(file)

    # Define the new censor condition
    new_censor = {
        'start': start,
        'end': end,
        'comment': comment
    }

    # Check if the censor condition already exists
    for row in data:
        #print(row)
        if row['module_id'] == pact_id:
            # Check if the censor condition is already present
            for censor in row['days_censored']:
                if (censor['start'] == new_censor['start'] and
                    censor['end'] == new_censor['end'] and
                    censor['comment'] == new_censor['comment']):
                    print(pact_id + ": Censored day condition already exists.")
                    return

            # If not found, add the new censor condition
            row['days_censored'].append(new_censor)
            print(pact_id + ": Censored day condition added.")

            # Save the updated data back to the file
            with open(file_path, 'w') as file:
                json.dump(data, file, indent=4)
            return

    print('check_and_add_censor_condition: Module ID not found.')

# Example usage
#check_and_add_censor_condition('path_to_your_file.json', 'P-0042-01', 
#    '2024-10-01', '2024-10-02', 'new monitoring system outage')

In [ ]:
def read_censored_days(censored_days_path):
    with open(censored_days_path, 'r') as file:
        censordata = pd.read_csv(file)
    for index, row in censordata.iterrows():
        pact_id = row['pact_id']
        start_date = row['start']
        end_date = row['end']
        comment = row['comment']
        
        # Check the module-metadata.json to see if this censordata is included
        if pact_id != 'site':
            module_metadata_path = '/Users/jsstein/bin/Box Sync/PACT - Data/' + pact_id[:6] + '-XX/Outdoor_SNL/data/metadata/module-metadata.json'
            check_and_add_censor_condition(module_metadata_path, pact_id, start_date, end_date, comment)
        else:
            # "site" in place of pact_id: Need to add to all modules active during period.
            ###### NEED TO UPDATE THIS SECTION (JSS:11/2/25)
            print('SITE Condition found')
            for index, row in active_modules.iterrows():
                pact_id = row['PACT_id']
                module_metadata_path = '/Users/jsstein/bin/Box Sync/PACT - Data/' + pact_id[:6] + '-XX/Outdoor_SNL/data/metadata/module-metadata.json'
                check_and_add_censor_condition(module_metadata_path, pact_id, start_date, end_date, comment)

In [1]:
# def process_module(df, pact_id, start_date, end_date):
       
#     #Filter out modules other than pact_id
#     dfmod = df.loc[df['ModuleID']==pact_id,:]
#     print(pact_id + ': Latest TmStamp = ' + df.index[-1].strftime('%Y-%m-%d %H:%M') + ' Testpad = ' + str(df['TestPad'].iloc[-1]) )
    
#     if df['TestPad'].iloc[-1] == 4: 
#         # TestPad 4: East Pact Tracker (PACTTracker1)
#         tablename = 'dbo.PACT_MET_PACTTracker1'
#         sql = f"SELECT TmStamp, E_Tracker1_Wm2_Avg, Trkr1Azimuth, Trkr1Altitude FROM {tablename} WHERE TmStamp BETWEEN '{start_date}' AND '{end_date}' ORDER BY TmStamp ASC"
#         df_tracker1 = pd.read_sql(sql, engine, index_col="TmStamp")
#         df_tracker1 = df_tracker1.rename(columns={'E_Tracker1_Wm2_Avg': 'poa_global', 'Trkr1Azimuth': 'surface_azimuth', 'Trkr1Altitude':'surface_tilt'})
#         df_tracker1 = df_tracker1.rename_axis('date_time')
#         df_tracker1.index = df_tracker1.index.tz_localize('MST')
        
#         #Reading PACT_MET_30s
#         tablename = 'dbo.PACT_MET_PACT_MET_30s'
#         sql = f"SELECT TmStamp,AmbientTemp_C_Avg  FROM {tablename} WHERE TmStamp BETWEEN '{start_date}' AND '{end_date}' ORDER BY TmStamp ASC"
#         df_met30s = pd.read_sql(sql, engine, index_col="TmStamp")
#         df_met30s = df_met30s.rename(columns={'AmbientTemp_C_Avg': 'temperature_air'})
#         df_met30s = df_met30s.rename_axis('date_time')
#         df_met30s.index = df_met30s.index.tz_localize('MST')

#         #Concat all columns in correct order
#         #print('TP=4:' + df_tracker1.columns + df_met30s.columns)
#         if 'df_tracker1' in locals():
#             df_tracker1 = df_tracker1[~df_tracker1.index.duplicated()]
#         df_met30s = df_met30s[~df_met30s.index.duplicated()]
#         df = df[~df.index.duplicated()]
#         df_all = pd.concat([df_tracker1['poa_global'],df_met30s['temperature_air'],df['temperature_module'],
#                     df['vmp'],df['imp'],df_tracker1['surface_tilt'],df_tracker1['surface_azimuth']], axis=1)
    
#     if df['TestPad'].iloc[-1] == 5: 
#         # TestPad 5: West Pact Tracker (PACTTracker2)
#         tablename = 'dbo.PACT_MET_PACTTracker2'
#         sql = f"SELECT TmStamp, E_Tracker2_Wm2_Avg, Trkr2Azimuth, Trkr2Altitude FROM {tablename} WHERE TmStamp BETWEEN '{start_date}' AND '{end_date}' ORDER BY TmStamp ASC"
#         df_tracker2 = pd.read_sql(sql, engine, index_col="TmStamp")
#         df_tracker2 = df_tracker2.rename(columns={'E_Tracker2_Wm2_Avg': 'poa_global', 'Trkr2Azimuth': 'surface_azimuth', 'Trkr2Altitude':'surface_tilt'})
#         df_tracker2 = df_tracker2.rename_axis('date_time')
#         df_tracker2.index = df_tracker2.index.tz_localize('MST')
        
#         #Reading PACT_MET_30s
#         tablename = 'dbo.PACT_MET_PACT_MET_30s'
#         sql = f"SELECT TmStamp,AmbientTemp_C_Avg  FROM {tablename} WHERE TmStamp BETWEEN '{start_date}' AND '{end_date}' ORDER BY TmStamp ASC"
#         df_met30s = pd.read_sql(sql, engine, index_col="TmStamp")
#         df_met30s = df_met30s.rename(columns={'AmbientTemp_C_Avg': 'temperature_air'})
#         df_met30s = df_met30s.rename_axis('date_time')
#         df_met30s.index = df_met30s.index.tz_localize('MST')

#         #Concat all columns in correct order
#         #print('TP=5:' + df_tracker2.columns + df_met30s.columns)
#         if 'df_tracker2' in locals():
#             df_tracker2 = df_tracker2[~df_tracker2.index.duplicated()]
#         df_met30s = df_met30s[~df_met30s.index.duplicated()]
#         df = df[~df.index.duplicated()]
#         df_all = pd.concat([df_tracker2['poa_global'],df_met30s['temperature_air'],df['temperature_module'],
#                     df['vmp'],df['imp'],df_tracker2['surface_tilt'],df_tracker2['surface_azimuth']], axis=1)
        
#         #####Adjust for bias caused by load switching#######
#         if pact_id == 'P-0042-03':
#             df_all['vmp'] = df_all['vmp']*1.14
#         if pact_id == 'P-0042-04':
#             df_all['vmp'] = df_all['vmp']*1.14

#     elif df['TestPad'].iloc[-1] == 6:
#         # TestPad 6: East Fixed Tilt

#         if int(start_date[0:4] + start_date[5:7])== 202512:
#             # Added logic to deal with moving POA out of PACT_MET table in December 2025.
#             tablename1 = 'dbo.PACT_MET_PACT_MET_30s'
#             tablename2 = 'dbo.PACT_MET_PACTWestTilt_30s'
#             sql = f"SELECT TmStamp,E_TiltPOA_Wm2_Avg,AmbientTemp_C_Avg  FROM {tablename1} WHERE TmStamp BETWEEN '{start_date}' AND '{end_date}' ORDER BY TmStamp ASC"
#             df_met30s_1 = pd.read_sql(sql, engine, index_col="TmStamp")
#             df_met30s_1.rename(columns={'E_TiltPOA_Wm2_Avg': 'E_WestTiltPOA_Wm2_Avg'}, inplace=True)
#             sql = f"SELECT TmStamp,E_WestTiltPOA_Wm2_Avg  FROM {tablename2} WHERE TmStamp BETWEEN '{start_date}' AND '{end_date}' ORDER BY TmStamp ASC"
#             df_met30s_2 = pd.read_sql(sql, engine, index_col="TmStamp")
#             #Merge tables
#             df_met30s = dfmet30s_1['E_WestTiltPOA_Wm2_Avg'].combine_first(df_met30s_2['E_WestTiltPOA_Wm2_Avg'])
#             df_met30s = df_met30s.rename(columns={'E_WestTiltPOA_Wm2_Avg': 'poa_global','AmbientTemp_C_Avg': 'temperature_air'})
#             df_met30s = df_met30s.rename_axis('date_time')
#             df_met30s.index = df_met30s.index.tz_localize('MST')
        
#         elif int(start_date[0:4] + start_date[5:7])< 202512:
#             # For all months BEFORE Dec 2025
#             tablename = 'dbo.PACT_MET_PACT_MET_30s'
#             sql = f"SELECT TmStamp,E_TiltPOA_Wm2_Avg,AmbientTemp_C_Avg  FROM {tablename} WHERE TmStamp BETWEEN '{start_date}' AND '{end_date}' ORDER BY TmStamp ASC"
#             df_met30s = pd.read_sql(sql, engine, index_col="TmStamp")
#             df_met30s = df_met30s.rename(columns={'E_TiltPOA_Wm2_Avg': 'poa_global','AmbientTemp_C_Avg': 'temperature_air'})
#             df_met30s = df_met30s.rename_axis('date_time')
#             df_met30s.index = df_met30s.index.tz_localize('MST')

#         elif int(start_date[0:4] + start_date[5:7])> 202512:
#             # For all months AFTER Dec 2025
#             #tablename1 = 'dbo.PACT_MET_PACT_MET_30s'
#             tablename2 = 'dbo.PACT_MET_PACTWestTilt_30s'
#             #sql = f"SELECT TmStamp,AmbientTemp_C_Avg  FROM {tablename1} WHERE TmStamp BETWEEN '{start_date}' AND '{end_date}' ORDER BY TmStamp ASC"
#             #df_met30s_1 = pd.read_sql(sql, engine, index_col="TmStamp")
#             sql = f"SELECT TmStamp,E_WestTiltPOA_Wm2_Avg  FROM {tablename2} WHERE TmStamp BETWEEN '{start_date}' AND '{end_date}' ORDER BY TmStamp ASC"
#             df_met30s_2 = pd.read_sql(sql, engine, index_col="TmStamp")
#             #Merge tables
#             #df_met30s = dfmet30s_1['E_EastTiltPOA_Wm2_Avg'].combine_first(df_met30s_2['E_EastTiltPOA_Wm2_Avg'])
#             df_met30s = df_met30s.rename(columns={'E_WestTiltPOA_Wm2_Avg': 'poa_global','AmbientTemp_C_Avg': 'temperature_air'})
#             df_met30s = df_met30s.rename_axis('date_time')
#             df_met30s.index = df_met30s.index.tz_localize('MST')
    
#         df_met30s['surface_tilt'] = np.zeros(len(df_met30s))+35
#         df_met30s['surface_azimuth'] = np.zeros(len(df_met30s))+180
    
#         #Concat all columns in correct order
#         #print('TP=6:' + df_met30s.columns)
#         if 'df_tracker2' in locals():
#             df_tracker2 = df_tracker2[~df_tracker2.index.duplicated()]
#         df_met30s = df_met30s[~df_met30s.index.duplicated()]
#         dfmod = dfmod[~dfmod.index.duplicated()]
#         df_all = pd.concat([df_met30s['poa_global'],df_met30s['temperature_air'],dfmod['temperature_module'],
#                     dfmod['vmp'],dfmod['imp'],df_met30s['surface_tilt'],df_met30s['surface_azimuth']], axis=1)
#     else:
#         print(pact_id + ': TestPad =' + str(dfmod['TestPad'][-1]) + ' not recognized')
    
        
#     path = '/Users/jsstein/bin/Box Sync/PACT - Data/'+ pact_id[:6] + '-XX/Outdoor_SNL/data/point-data/'
#     df_all.to_csv(path +'point-data_' + pact_id + '_' + yearmonth + '.csv', index=True)
    
#     # # Write file to BOX
#     # client = make_client()
#     # folders = get_key_folders(pact_id[0:6], client)
#     # datafile = path + 'point-data_' + pact_id + '_' + str(start_date)[0:7] + '.csv'
#     # upload_file(datafile, folders['point-data'], client)
#     # print(datafile + ' copied to Box')

#     #Write file to AWS S3
#     s3_key = pact_id[:6] + '-XX/Outdoor_SNL/data/point-data/' + 'point-data_' + pact_id + '_' + yearmonth + '.csv'
#     bucket.upload_file(path + 'point-data_' + pact_id + '_' + str(start_date)[0:7] + '.csv' ,s3_key)
#     print(s3_key + ' copied to AWS S3')

In [5]:
start_date = '2025-12-01'
int(start_date[0:4] + start_date[5:7])

202512